In [1]:
import pandas as pd
from tqdm import tqdm
lyrics = pd.read_csv('song_lyrics_lemmatized.csv')

In [2]:
lyrics_corpus_tokenized = list()
for song in tqdm(lyrics['tokenized, lemmatized, no bad words'], desc="writing to the list"):
    lyrics_corpus_tokenized.append(song.strip("[]").replace("'", "").split(", "))
print(type(lyrics_corpus_tokenized[0]))
print(lyrics_corpus_tokenized[0])

writing to the list: 100%|██████████| 5334078/5334078 [00:38<00:00, 139469.30it/s]

<class 'list'>
['watchin', 'every', 'watchin', 'closely', 'butter', 'bread', 'wanna', 'toast', 'keep', 'head', 'supposed', 'sidetracked', 'clapped', 'close', 'foot', 'sleep', 'tired', 'feel', 'wired', 'like', 'codeine', 'day', 'brother', 'gotta', 'admire', 'four', 'fiend', 'away', 'pain', 'wish', 'quick', 'see', 'sellin', 'caine', 'til', 'brain', 'fried', 'fricassee', 'lie', 'time', 'never', 'bothered', 'bar', 'gettin', 'properly', 'squad', 'lack', 'respect', 'authority', 'laughin', 'hard', 'happy', 'escapin', 'poverty', 'however', 'brief', 'know', 'game', 'valley', 'peak', 'expectation', 'dip', 'precipitation', 'stack', 'chip', 'hardly', 'youth', 'used', 'soon', 'see', 'mill', 'big', 'willie', 'game', 'ha', 'grown', 'prefer', 'call', 'william', 'illin', 'revenue', 'rayful', 'edmond', 'like', 'channel', 'news', 'round', 'seven', 'jewel', 'head', 'dead', 'mic', 'forgettin', 'ever', 'knew', 'convenient', 'amnesia', 'suggest', 'call', 'lawyer', 'know', 'procedure', 'lock', 'body', 'trap',

In [3]:
words_to_remove = {'verse', 'chorus', 'hook', 'bridge', 'intro', 'outro', 'pre-chorus', 'pre-hook', 'get', 'got'}


In [4]:
lyrics_corpus_tokenized = [
    [token for token in song if token not in words_to_remove]
    for song in tqdm(lyrics_corpus_tokenized, desc="removing")
]

removing: 100%|██████████| 5334078/5334078 [00:32<00:00, 165134.52it/s]


In [5]:
from gensim.corpora import Dictionary
from tqdm import tqdm
import random

# Assuming you have a big list called 'big_list'
  # Number of items to choose

# Randomly select 20,000 items from the big list
# lyric_corpus_tokenized = lyric_corpus_tokenized.tolist()

tqdm_corpus = tqdm(lyrics_corpus_tokenized, desc="Creating dictionary")
dictionary = Dictionary(tqdm_corpus)

Creating dictionary: 100%|██████████| 5334078/5334078 [02:11<00:00, 40673.05it/s]


In [6]:
dictionary.filter_extremes(no_below=100, no_above=0.8)

In [7]:
from gensim.corpora import MmCorpus
gensim_corpus = []
with tqdm(total=len(lyrics_corpus_tokenized), desc="Creating Gensim Corpus") as pbar:
    for song in lyrics_corpus_tokenized:
        doc = dictionary.doc2bow(song)
        gensim_corpus.append(doc)
        pbar.update(1)
temp = dictionary[0]
id2word = dictionary.id2token

Creating Gensim Corpus: 100%|██████████| 5334078/5334078 [01:24<00:00, 62870.07it/s] 


In [29]:
import time
from gensim.models import LdaModel
import random

start_time = time.time()
chunksize = 2000
passes = 20
iterations = 400
num_topics = 4
sample_size = 500000


gensim2 = random.sample(gensim_corpus, sample_size)

lda_model = LdaModel(
    corpus=gensim2,
    id2word=id2word,
    chunksize=chunksize,
    alpha='auto',
    eta='auto',
    iterations=iterations,
    num_topics=num_topics,
    passes=passes
)

end_time = time.time()
training_time = end_time - start_time
print("Training time: {:.2f} seconds".format(training_time))

Training time: 1124.63 seconds


In [30]:
import time
from gensim.models import LdaModel
import random
# lda_model = LdaModel.load("lda_model_topics_full")

In [34]:
# import pyLDAvis
# import pyLDAvis.gensim_models as gensimvis
# vis_data = gensimvis.prepare(lda_model, gensim_corpus, dictionary)
# pyLDAvis.display(vis_data)

In [32]:

pyLDAvis.save_html(vis_data, './Lyrics_LDA_'+ '4' +'.html')

In [33]:
lda_model.save("lda_model_4")


In [ ]:
topics = ['life and relationships', 'money and authority', 'general', 'religion and society']

In [ ]:
predicted_topics = list()
with tqdm(total=len(gensim_corpus), desc="Predicting topics") as pbar:
    for song in gensim_corpus:
        topic_dist = lda_model.get_document_topics(song)
        predicted_topics.append(topic_dist)
        pbar.update(1)

Predicting topics:  44%|████▍     | 2360193/5334078 [04:23<05:14, 9450.68it/s] 

In [ ]:
max_indices = []
for vector in predicted_topics:
    max_tuple = max(vector, key=lambda t: t[1])
    max_indices.append(topics[max_tuple[0]])


In [ ]:
print(max_indices[:10])

In [ ]:
lyrics['topic'] = max_indices

In [ ]:
# count = lyrics['predicted topics'].value_counts()['family and religion']
# print(count, " family and religion")

count = lyrics['topic'].value_counts()['life and relationships']
print(count, " life and relationships")
count = lyrics['topic'].value_counts()['money and authority']
print(count, " money and authority")
count = lyrics['topic'].value_counts()['general']
print(count, " general")
count = lyrics['topic'].value_counts()['religion and society']
print(count, " religion and society")


In [ ]:
lyrics['topic vectors'] = predicted_topics

In [ ]:
lyrics.to_csv('song_topics.csv', index=False)